In [1]:
import pandas as pd
import requests
import json

## Gets a list of all the occurrence datasets published in GBIF

By using the registry and the occurrence API, this scripst get a list of all the occurrence datasets published on GBIF and get the number of occurrences they contain.
Running the script might take a while.

In [2]:
GBIF_API = "http://api.gbif.org/v1/"
root_URL = "https://www.gbif.org/dataset/"
step = 500

In [5]:
datasets_with_count = pd.DataFrame()

endOfRecords = False
offset = 0
while not endOfRecords:
    param = {
        "offset": offset,
        "limit": step,
        "type": "OCCURRENCE"
    }
    # Get dataset
    response = requests.get(GBIF_API + "dataset/search", param)
    if response.ok:
        dataset_list = response.json()
        for dataset in dataset_list["results"]:
            datasets_with_count.at[dataset["key"], "URL"] = root_URL + dataset["key"]
            datasets_with_count.at[dataset["key"], "title"] = dataset["title"]
            if "description" in dataset:
                datasets_with_count.at[dataset["key"], "description"] = dataset["description"]
            
            # Get occurrence count
            response_occ_count = requests.get(GBIF_API + "occurrence/count", {"datasetKey": dataset["key"]})
            if response_occ_count.ok:
                datasets_with_count.at[dataset["key"], "count_occ"] = response_occ_count.json()
            else:
                print(response_occ_count)
                print(dataset["key"])
        offset += step
        endOfRecords = dataset_list["endOfRecords"]
    else:
        print(response)
        endOfRecords = True

In [8]:
datasets_with_count.to_csv('occurrence_datasets_with_count.txt', sep = '\t', index=False)

In [7]:
datasets_with_count

,URL,title,description,count_occ
8a863029-f435-446a-821e-275f4f641165,https://www.gbif.org/dataset/8a863029-f435-446...,"Observation.org, Nature data from the Netherlands",This dataset contains occurrence data of flora...,26859363.0
75956ee6-1a2b-4fa3-b3e8-ccda64ce6c2d,https://www.gbif.org/dataset/75956ee6-1a2b-4fa...,INPN - Données flore des CBN agrégées par la FCBN,Dès la création de leur Fédération (FCBN - www...,20999334.0
e6fab7b3-c733-40b9-8df3-2a03e49532c1,https://www.gbif.org/dataset/e6fab7b3-c733-40b...,Flora von Deutschland (Phanerogamen),Verbreitung der Farn- und Bluetenpflanzen in D...,4416438.0
6ac3f774-d9fb-4796-b3e9-92bf6c81c084,https://www.gbif.org/dataset/6ac3f774-d9fb-479...,naturgucker,naturgucker.de / enjoynature.net is a rapidly ...,9918225.0
f06fef3c-6ea6-4345-b724-a1b8e490dc55,https://www.gbif.org/dataset/f06fef3c-6ea6-434...,Anillamiento SEO_Bird ringing,Scientific bird ringing is a research method b...,8957407.0
...,...,...,...,...
f8ceb4e6-82ff-4325-afc8-eb5e64b5f842,https://www.gbif.org/dataset/f8ceb4e6-82ff-432...,Ecosystem biomonitoring with eDNA: metabarcodi...,Effective marine management requires comprehen...,546.0
02abb9d1-7d81-42b3-ac9a-3b3d0c7a5280,https://www.gbif.org/dataset/02abb9d1-7d81-42b...,Fungimap,This dataset contains observational records of...,162588.0
f8f683a8-ab6c-4c4b-a6ec-53b23d5404d0,https://www.gbif.org/dataset/f8f683a8-ab6c-4c4...,Reef Life Survey: Cryptic Fish,Reef Life Survey is designed to develop and re...,36396.0
71315e6b-727f-4e71-91db-8823fe06ba22,https://www.gbif.org/dataset/71315e6b-727f-4e7...,Royal Ontario Museum - Entomology (ROME),"Vouchered occurrence records for arthropods, p...",4050.0
